In [100]:
import dask.dataframe as dd
import dask
import re

In [101]:
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'case:concept:name': 'int64',
    'dueDate': 'object',
    'case:termName': 'string',
    'dateStop': 'object',
    'case:endDate': 'object',
    'case:endDatePlanned': 'object',
    'case:parts': 'object',
    'msgCode': 'string',
    'msgType': 'string',
    'case:landRegisterID': 'object'
}

# list of file paths to be loaded

file_paths = ['BPIC15_1', 'BPIC15_2', 'BPIC15_3', 'BPIC15_4', 'BPIC15_5']

# load the first file as a Dask dataframe
df = dd.read_csv('../project/data/{}.csv'.format(file_paths[0]), dtype=columnTypes, encoding="ISO-8859-1", sep=',')

# iterate over the remaining files
for file_path in file_paths[1:]:
    # usecols parameter to load only the columns that are present in both dataframes
    df_temp = dd.read_csv('../project/data/{}.csv'.format(file_path), dtype=columnTypes)
    # concatenate the dataframes along the rows
    df = dd.concat([df, df_temp], interleave_partitions=True)

# columnTypes = {
#     'OfferID': 'string'
# }


# BPI 2014 sep=''
# df = df.rename(columns={"Incident ID": "case:concept:name", "IncidentActivity_Type": "concept:name", "DateStamp": "time:timestamp"})

# df = df.rename(columns={"case concept:name": "case:concept:name", "event concept:name": "concept:name", "event time:timestamp": "time:timestamp"})
for column in df.columns:
    if re.search("[Dd]ate.*|time.*", column):
        df[column] = dask.dataframe.to_datetime(df[column], utc=True)

df['case:concept:name'] = df['case:concept:name'].replace(to_replace="[a-zA-Z]", value='', regex=True)
df['case:concept:name'] = df['case:concept:name'].astype('int')
        
df = df.repartition(npartitions=1)

In [102]:
indexed_df = df.set_index('case:concept:name', drop=False)
indexed_df['case:concept:name'] = indexed_df['case:concept:name'].astype({'case:concept:name': 'string'})
del df

In [ ]:
def getAverageEventsPerCase(partition):
    return int(partition.groupby('case:concept:name')['concept:name'].count().mean())

In [103]:
indexed_df.index = indexed_df.index.rename('caseId')
indexed_df = indexed_df.repartition(npartitions=6)

In [104]:
# cases count per partition
# partition_1 = indexed_df.repartition(npartitions=1).map_partitions(lambda partition: partition[['case:concept:name']].describe())

In [105]:
# average events per case per partition
# partition_1 = indexed_df.repartition(npartitions=1).map_partitions(getAverageEventsPerCase)

In [106]:
indexed_df['case:concept:name'].describe().compute()

unique        5647
count       262628
top       11550160
freq           154
Name: case:concept:name, dtype: object

In [107]:
partition_1 = indexed_df.repartition(npartitions=1).map_partitions(lambda partition: partition[['case:concept:name']].describe())

In [108]:
partition_2 = indexed_df.repartition(npartitions=2).map_partitions(lambda partition: partition[['case:concept:name']].describe())

In [109]:
partition_4 = indexed_df.repartition(npartitions=4).map_partitions(lambda partition: partition[['case:concept:name']].describe())

In [110]:
partition_6 = indexed_df.repartition(npartitions=6).map_partitions(lambda partition: partition[['case:concept:name']].describe())

In [111]:
partition_6.compute()

,case:concept:name
count,128963
unique,2993
top,4032873
freq,124
count,78699
unique,1538
top,9098835
freq,134
count,28155
unique,627


In [99]:
partition_2.memory_usage_per_partition(index=True, deep=False).compute()

0    64
1    64
dtype: int64

In [64]:
# partition_1.to_csv('bpi_2015_cases_count_frequency/1_partition/*.csv')

In [68]:
# partition_2.to_csv('bpi_2015_cases_count_frequency/2_partition/*.csv')

In [69]:
# partition_4.to_csv('bpi_2015_cases_count_frequency/4_partition/*.csv')

In [70]:
# partition_6.to_csv('bpi_2015_cases_count_frequency/6_partition/*.csv')

In [71]:
# x = df.groupby('case:concept:name')['concept:name'].count().value_counts()

In [72]:
# output = x.to_csv('bpi_2015_events_count_frequency.csv', single_file=True)